# Traffic model research

In [1]:
from src import traffic_model as model
from joblib import Parallel, delayed
import numpy as np

### Research human drive vehicle

__Single line__

In [33]:
road_param = {
    "N_cells": 1000,
    "N_lane": 1,
}
cars = {model.HumanDriveVehicle: 200}

In [34]:
model_1 = model.Model(road_parametrs=road_param, vehicles=cars)

In [35]:
%%time
model_1.model_stabilization(1000)

CPU times: total: 22.9 s
Wall time: 22.9 s


In [36]:
model_1.model_research(100)

In [38]:
model_1.result

{'rho': 0.2, 'flow': array([0.49995]), 'change_frequency': 0.0}

__Three line__

In [2]:
road_param = {
    "N_cells": 1000,
    "N_lane": 3,
}
cars = {model.HumanDriveVehicle: 600}

In [3]:
model_3 = model.Model(road_parametrs=road_param, vehicles=cars)

In [4]:
%%time
model_3.model_stabilization(1000)

C:\Users\Maksim\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


CPU times: total: 1min 18s
Wall time: 1min 18s


In [5]:
model_3.model_research(100)

In [7]:
model_3.result

{'rho': 0.19999999999999998,
 'flow': array([0.49589]),
 'change_frequency': 0.003333333333333333}